In [2]:
import requests
from pathlib import Path

# Get the file path
output_folder = "documents"
filename = "think_python_guide.pdf"
url = "https://greenteapress.com/thinkpython/thinkpython.pdf"
file_path = Path(output_folder) / filename


def download_file(url: str, file_path: Path):
    response = requests.get(url, stream=True, timeout=30)
    response.raise_for_status()
    file_path.write_bytes(response.content)


# Download the file if it doesn't exist
if not file_path.exists():
    download_file(
        url=url,
        file_path=file_path,
    )

In [3]:
from markitdown import MarkItDown

# Initialize the converter
md = MarkItDown()

# Convert the Python guide to markdown
result = md.convert(file_path)
python_guide_content = result.text_content

# Display the conversion results
print("First 300 characters:")
print(python_guide_content[:300] + "...")

First 300 characters:
Think Python

How to Think Like a Computer Scientist

Version 2.0.17

Think Python

How to Think Like a Computer Scientist

Version 2.0.17

Allen Downey

Green Tea Press

Needham, Massachusetts

Copyright © 2012 Allen Downey.

Green Tea Press
9 Washburn Ave
Needham MA 02492

Permission is granted...


In [4]:
# Organize the converted document
processed_document = {
    'source': file_path,
    'content': python_guide_content
}

# Create a list containing our single document for consistency with downstream processing
documents = [processed_document]

# Document is now ready for chunking and embedding
print(f"Document ready: {len(processed_document['content']):,} characters")

Document ready: 460,251 characters


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

c:\Users\danya\Desktop\projects\rag_pipline\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Original: 336 chars → 3 chunks
Chunk 1: Machine learning transforms data processing. It enables pattern recognition without explicit programming.
Chunk 2: Deep learning uses neural networks with multiple layers. These networks discover complex patterns automatically.
Chunk 3: Natural language processing combines ML with linguistics. It helps computers understand human language effectively.


In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,         # Optimal chunk size for Q&A scenarios
    chunk_overlap=120,      # 20% overlap to preserve context
    separators=["\n\n", "\n", ". ", " ", ""]  # Split hierarchy
)

In [7]:
def process_document(doc, text_splitter):
    """Process a single document into chunks."""
    doc_chunks = text_splitter.split_text(doc["content"])
    return [{"content": chunk, "source": doc["source"]} for chunk in doc_chunks]


# Process all documents and create chunks
all_chunks = []
for doc in documents:
    doc_chunks = process_document(doc, text_splitter)
    all_chunks.extend(doc_chunks)

In [8]:
from collections import Counter

source_counts = Counter(chunk["source"] for chunk in all_chunks)
chunk_lengths = [len(chunk["content"]) for chunk in all_chunks]

print(f"Total chunks created: {len(all_chunks)}")
print(f"Chunk length: {min(chunk_lengths)}-{max(chunk_lengths)} characters")
print(f"Source document: {Path(documents[0]['source']).name}")

Total chunks created: 1007
Chunk length: 68-598 characters
Source document: think_python_guide.pdf


In [10]:
from sentence_transformers import SentenceTransformer

# Load Q&A-optimized embedding model (downloads automatically on first use)
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

# Extract documents and create embeddings
documents = [chunk["content"] for chunk in all_chunks]
embeddings = model.encode(documents)

print(f"Embedding generation results:")
print(f"  - Embeddings shape: {embeddings.shape}")
print(f"  - Vector dimensions: {embeddings.shape[1]}")

c:\Users\danya\Desktop\projects\rag_pipline\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danya\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not i

Embedding generation results:
  - Embeddings shape: (1007, 768)
  - Vector dimensions: 768


In [12]:
# Test how one query finds relevant Python programming content
from sentence_transformers import util

query = "How do you define functions in Python?"
document_chunks = [
    "Variables store data values that can be used later in your program.",
    "A function is a block of code that performs a specific task when called.",
    "Loops allow you to repeat code multiple times efficiently.",
    "Functions can accept parameters and return values to the calling code."
]

# Encode query and documents
query_embedding = model.encode(query)
doc_embeddings = model.encode(document_chunks)

In [13]:
# Calculate similarities using SentenceTransformers util
similarities = util.cos_sim(query_embedding, doc_embeddings)[0]

# Create ranked results
ranked_results = sorted(
    zip(document_chunks, similarities), 
    key=lambda x: x[1], 
    reverse=True
)

print(f"Query: '{query}'")
print("Document chunks ranked by relevance:")
for i, (chunk, score) in enumerate(ranked_results, 1):
    print(f"{i}. ({score:.3f}): '{chunk}'")

Query: 'How do you define functions in Python?'
Document chunks ranked by relevance:
1. (0.674): 'A function is a block of code that performs a specific task when called.'
2. (0.607): 'Functions can accept parameters and return values to the calling code.'
3. (0.461): 'Loops allow you to repeat code multiple times efficiently.'
4. (0.448): 'Variables store data values that can be used later in your program.'


In [17]:
import chromadb

# Create persistent client for data storage
client = chromadb.PersistentClient(path="./chroma_db")

# Create collection for business documents (or get existing)
collection = client.get_or_create_collection(
    name="python_guide",
    metadata={"description": "Python programming guide"}
)

print(f"Created collection: {collection.name}")
print(f"Collection ID: {collection.id}")

Created collection: python_guide
Collection ID: 25bd03d2-dbbd-4127-be76-fdcba4767ee7


In [18]:
# Prepare metadata and add documents to collection
metadatas = [{"document": Path(chunk["source"]).name} for chunk in all_chunks]

collection.add(
    documents=documents,
    embeddings=embeddings.tolist(), # Convert numpy array to list
    metadatas=metadatas, # Metadata for each document
    ids=[f"doc_{i}" for i in range(len(documents))], # Unique identifiers for each document
)

print(f"Collection count: {collection.count()}")

Collection count: 1007


In [19]:
def format_query_results(question, query_embedding, documents, metadatas):
    """Format and print the search results with similarity scores"""
    from sentence_transformers import util

    print(f"Question: {question}\n")

    for i, doc in enumerate(documents):
        # Calculate accurate similarity using sentence-transformers util
        doc_embedding = model.encode([doc])
        similarity = util.cos_sim(query_embedding, doc_embedding)[0][0].item()
        source = metadatas[i].get("document", "Unknown")

        print(f"Result {i+1} (similarity: {similarity:.3f}):")
        print(f"Document: {source}")
        print(f"Content: {doc[:300]}...")
        print()


def query_knowledge_base(question, n_results=2):
    """Query the knowledge base with natural language"""
    # Encode the query using our SentenceTransformer model
    query_embedding = model.encode([question])

    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=n_results,
        include=["documents", "metadatas", "distances"],
    )

    # Extract results and format them
    documents = results["documents"][0]
    metadatas = results["metadatas"][0]

    format_query_results(question, query_embedding, documents, metadatas)

In [24]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate

# Initialize the local LLM
llm = OllamaLLM(model="llama3.2:latest", temperature=0.1)

In [25]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a Python programming expert. Based on the provided documentation, answer the question clearly and accurately.

Documentation:
{context}

Question: {question}

Answer (be specific about syntax, keywords, and provide examples when helpful):"""
)

# Create the processing chain
chain = prompt_template | llm

In [26]:
def retrieve_context(question, n_results=5):
    """Retrieve relevant context using embeddings"""
    query_embedding = model.encode([question])
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=n_results,
        include=["documents", "metadatas", "distances"],
    )

    documents = results["documents"][0]
    context = "\n\n---SECTION---\n\n".join(documents)
    return context, documents


def get_llm_answer(question, context):
    """Generate answer using retrieved context"""
    answer = chain.invoke(
        {
            "context": context[:2000],
            "question": question,
        }
    )
    return answer


def format_response(question, answer, source_chunks):
    """Format the final response with sources"""
    response = f"**Question:** {question}\n\n"
    response += f"**Answer:** {answer}\n\n"
    response += "**Sources:**\n"

    for i, chunk in enumerate(source_chunks[:3], 1):
        preview = chunk[:100].replace("\n", " ") + "..."
        response += f"{i}. {preview}\n"

    return response


def enhanced_query_with_llm(question, n_results=5):
    """Query function combining retrieval with LLM generation"""
    context, documents = retrieve_context(question, n_results)
    answer = get_llm_answer(question, context)
    return format_response(question, answer, documents)

In [27]:
enhanced_response = enhanced_query_with_llm("How do if-else statements work in Python?")
print(enhanced_response)

**Question:** How do if-else statements work in Python?

**Answer:** If-else statements in Python are used for conditional execution of code. Here's a breakdown of how they work:

**Syntax**

The basic syntax of an if-else statement in Python is as follows:
```
if condition:
    # code to execute if condition is true
elif condition2:
    # code to execute if condition1 is false and condition2 is true
else:
    # code to execute if both conditions are false
```
**Keywords**

The keywords used in an if-else statement are:

* `if`: used to check a condition
* `elif` (short for "else if"): used to check another condition if the first one is false
* `else`: used to specify code to execute if all conditions are false

**How it works**

Here's how an if-else statement works in Python:

1. The interpreter evaluates the condition inside the `if` block.
2. If the condition is true, the code inside the `if` block is executed.
3. If the condition is false, the interpreter moves on to the next line

In [28]:
def stream_llm_answer(question, context):
    """Stream LLM answer generation token by token"""
    for chunk in chain.stream({
        "context": context[:2000],
        "question": question,
    }):
        yield getattr(chunk, "content", str(chunk))

In [29]:
import time

# Test the streaming functionality
question = "What are Python loops?"
context, documents = retrieve_context(question, n_results=3)

print("Question:", question)
print("Answer: ", end="", flush=True)

# Stream the answer token by token
for token in stream_llm_answer(question, context):
    print(token, end="", flush=True)
    time.sleep(0.05)

Question: What are Python loops?
Answer: Python loops are control structures that allow you to execute a block of code repeatedly for each item in a sequence. There are two main types of loops in Python:

1. **For Loops**

A for loop is used to iterate over a sequence (such as a list, tuple, or string) and perform an action on each item in the sequence.

Syntax:
```python
for variable in sequence:
    # code to be executed
```
Example:
```python
fruits = ['apple', 'banana', 'cherry']
for fruit in fruits:
    print(fruit)
```
Output:
```
apple
banana
cherry
```
In this example, the `for` loop iterates over the `fruits` list and assigns each item to the variable `fruit`. The code inside the loop is executed for each item in the sequence.

2. **While Loops**

A while loop is used to execute a block of code repeatedly as long as a certain condition is true.

Syntax:
```python
while condition:
    # code to be executed
```
Example:
```python
i = 0
while i < 5:
    print(i)
    i += 1
```
Ou

In [30]:
import gradio as gr


def rag_interface(question):
    """Gradio interface reusing existing format_response function"""
    if not question.strip():
        yield "Please enter a question."
        return

    # Use modular retrieval and streaming
    context, documents = retrieve_context(question, n_results=5)

    response_start = f"**Question:** {question}\n\n**Answer:** "
    answer = ""

    # Stream the answer progressively
    for token in stream_llm_answer(question, context):
        answer += token
        yield response_start + answer

    # Use existing formatting function for final response
    yield format_response(question, answer, documents)

In [33]:
# Create Gradio interface with streaming support
demo = gr.Interface(
    fn=rag_interface,
    inputs=gr.Textbox(
        label="Ask a question about Python programming",
        placeholder="How do if-else statements work in Python?",
        lines=2,
    ),
    outputs=gr.Markdown(label="Answer"),
    title="Intelligent Document Q&A System",
    description="Ask questions about Python programming concepts and get instant answers with source citations.",
    examples=[
        "How do if-else statements work in Python?",
        "What are the different types of loops in Python?",
        "How do you handle errors in Python?",
    ],
    # allow_flagging="never"
)

# Launch the interface with queue enabled for streaming
if __name__ == "__main__":
    demo.queue().launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://bb4bb954556591c7a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Created dataset file at: .gradio\flagged\dataset1.csv
